## Generation Probabilities Matrix

We will do random walks from each document, using the graph represented by the sparsified Similarity Matrix.

Generation probability from document i to document j is an approximation of semantic similarity between the two documents. We execute n random walks between the two nodes, varying the number of hops (t) from 1 to 3. The resulting generation probability q<sub>i,j</sub> is computed empirically, given a sufficiently large number of walks, from the following equation.

<a href="https://www.codecogs.com/eqnedit.php?latex=\fn_jvn&space;q_{ij}&space;=&space;\frac{N(i,&space;j)}{\sum_{x&space;\epsilon&space;J}&space;N(i,&space;x)}" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\fn_jvn&space;q_{ij}&space;=&space;\frac{N(i,&space;j)}{\sum_{x&space;\epsilon&space;J}&space;N(i,&space;x)}" title="q_{ij} = \frac{N(i, j)}{\sum_{x \epsilon J} N(i, x)}" /></a>

For our purposes, we will choose the number of walks from each node as c=80 (same as our top generators constant). For t &gt; 1, since at each stage we will have to do c hops from each source, we set the number of walks as $\sqrt[t]{c}$.

Further, the generation probability is tempered by the number of hops, so documents reachable via 3 hops have lesser influence on the generation probability than documents reachable directly (i.e., via a single hop).

<a href="https://www.codecogs.com/eqnedit.php?latex=\fn_jvn&space;gen^t(d_i|d_j)&space;=&space;\frac{\sum_{s=1}^t&space;q_{ij}}{t}" target="_blank"><img src="https://latex.codecogs.com/png.latex?\fn_jvn&space;gen^t(d_i|d_j)&space;=&space;\frac{\sum_{s=1}^t&space;q_{ij}}{t}" title="gen^t(d_i|d_j) = \frac{\sum_{s=1}^t q_{ij}}{t}" /></a>


In [1]:
import numpy as np
import os

In [2]:
DATA_DIR = "../data"
SIM_MATRIX_PATH = os.path.join(DATA_DIR, "cosim.npy")
NUM_TOP_GENERATORS = 80

GEN_MATRIX_PATH_TEMPLATE = os.path.join(DATA_DIR, "genprobs_{:d}.npy")

### Generating Random Walks

We want to generate random walks of specific length, starting from each node in the graph. The walks would choose from a list of reachable nodes, where the reachability is dictated by transition probabilities.

Neither Spark Graphframes and Neo4j offer a built-in algorithm to construct random walks on weighted graphs. It is possible to construct a naive version which will iterate over paths starting from each node in the graph. 

However, constructing random walk directly on the adjacency matrix using the function below gives much better performance. Given an array of source nodes, the function below will construct paths of length 1 starting with each source node in the input.

Path lengths &gt; 1 can be constructed by running this repeatedly in a loop, as shown below.

In [3]:
def random_walk(paths, id_matrix, prob_matrix, c, num_hops):
    input_paths = []
    input_paths.extend(paths)
    for i in range(num_hops):
        output_paths = []
        num_choices = int(np.round(np.power(c, 1/num_hops)))
        for path in input_paths:
            # print("path:", path)
            source = path[-1]
            if np.sum(prob_matrix[source]) == 0:
                # skip this path, nothing is reachable from node
                continue
            try:
                choices = np.random.choice(id_matrix[source, :], 
                    size=num_choices, 
                    p=prob_matrix[source, :])
            except ValueError as e:
                print("source:", source)
                print(prob_matrix[source, :].tolist())
                raise e
            for choice in choices:
                output_path = []
                output_path.extend(path)
                output_path.append(choice)
                output_paths.append(output_path)
        input_paths = []
        input_paths.extend(output_paths)
    return output_paths


# test
M = np.zeros((10, 10))
for i in range(30):
    i = np.random.randint(0, high=10)
    j = np.random.randint(0, high=10)
    M[i, j] = float(np.random.randint(0, high=10))
M_rowsum = np.sum(M, axis=1).reshape(M.shape[0], 1)
M_rowsum[M_rowsum == 0] = 0.1
M_prob = M / M_rowsum

M_id = np.array([np.arange(10)] * M.shape[0]).reshape(M.shape)

input_paths = [[x] for x in np.arange(10)]
output_paths = random_walk(input_paths, M_id, M_prob, 5, 2)
print("inputs:", input_paths)
print("outputs:", output_paths)

inputs: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
outputs: [[0, 8, 9], [0, 8, 4], [0, 3, 9], [0, 3, 5], [1, 9, 5], [1, 9, 1], [1, 9, 1], [1, 9, 5], [2, 3, 5], [2, 3, 9], [2, 3, 5], [2, 3, 9], [3, 5, 4], [3, 5, 4], [3, 9, 1], [3, 9, 1], [4, 4, 0], [4, 4, 4], [4, 4, 0], [4, 4, 4], [5, 4, 0], [5, 4, 0], [5, 4, 0], [5, 4, 4], [6, 2, 3], [6, 2, 3], [6, 2, 3], [6, 2, 3], [7, 5, 4], [7, 5, 4], [7, 6, 2], [7, 6, 2], [8, 4, 0], [8, 4, 0], [8, 4, 4], [8, 4, 4], [9, 5, 4], [9, 5, 4], [9, 1, 3], [9, 1, 3]]


### Generation Probability Matrix

The set of random walks generated is used to compute the generation probability matrix. For multi-hop walks, we compute empirical probabilities and apply the path length discount in-place in the function below.

In [4]:
def compute_generation_probability(paths, num_nodes, num_hops):
    G = np.zeros((num_nodes, num_nodes))
    for i in range(num_hops):
        for path in paths:
            source = path[0]
            target = path[i+1]
            G[source, target] += (1 / num_hops)
    G_rowsum = np.sum(G, axis=1).reshape(G.shape[0], 1)
    G_rowsum[G_rowsum == 0] = 1e-19
    return G / G_rowsum

# test
Q = compute_generation_probability(output_paths, M.shape[0], 2)
print(Q)

[[0.    0.    0.    0.25  0.125 0.125 0.    0.    0.25  0.25 ]
 [0.    0.25  0.    0.    0.    0.25  0.    0.    0.    0.5  ]
 [0.    0.    0.    0.5   0.    0.25  0.    0.    0.    0.25 ]
 [0.    0.25  0.    0.    0.25  0.25  0.    0.    0.    0.25 ]
 [0.25  0.    0.    0.    0.75  0.    0.    0.    0.    0.   ]
 [0.375 0.    0.    0.    0.625 0.    0.    0.    0.    0.   ]
 [0.    0.    0.5   0.5   0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.25  0.    0.25  0.25  0.25  0.    0.    0.   ]
 [0.25  0.    0.    0.    0.75  0.    0.    0.    0.    0.   ]
 [0.    0.25  0.    0.25  0.25  0.25  0.    0.    0.    0.   ]]


### Random Walk on Document Graph

We retrieve the transition probability matrix of sparsified cosine similarities S and construct an ID-matrix from it (this is only for ease of generating path sequences). We then construct the initial set of paths (length 0) consisting of all the source nodes.

In [5]:
prob_matrix = np.load(SIM_MATRIX_PATH)
np.nan_to_num(prob_matrix, copy=False)

id_matrix = (np.concatenate(
    [np.arange(prob_matrix.shape[1])] * prob_matrix.shape[0])
    .reshape(prob_matrix.shape))

print(prob_matrix.shape, id_matrix.shape)

(18810, 18810) (18810, 18810)


In [6]:
input_paths = [[x] for x in np.arange(id_matrix.shape[0])]
print("number of input paths:", len(input_paths))

number of input paths: 18810


### Random Walk and Gen.Probs for t=1

In [7]:
output_paths_1 = random_walk(
    input_paths, id_matrix, prob_matrix, NUM_TOP_GENERATORS, 1)
print("number of output paths of length 1:", len(output_paths_1))
gen_probs_1 = compute_generation_probability(
    output_paths_1, id_matrix.shape[0], 1)
print(gen_probs_1.shape)
np.save(GEN_MATRIX_PATH_TEMPLATE.format(1), gen_probs_1)

number of output paths of length 1: 1504160
(18810, 18810)


### Random Walk and Gen.Probs for t=2

In [8]:
output_paths_2 = random_walk(
    input_paths, id_matrix, prob_matrix, NUM_TOP_GENERATORS, 2)
print("number of output paths of length 2:", len(output_paths_2))
gen_probs_2 = compute_generation_probability(
    output_paths_2, id_matrix.shape[0], 2)
print(gen_probs_2.shape)
np.save(GEN_MATRIX_PATH_TEMPLATE.format(2), gen_probs_2)

number of output paths of length 2: 1522962
(18810, 18810)


### Random Walk and Gen.Probs for t=3

In [9]:
output_paths_3 = random_walk(
    input_paths, id_matrix, prob_matrix, NUM_TOP_GENERATORS, 3)
print("number of output paths of length 3:", len(output_paths_3))
gen_probs_3 = compute_generation_probability(
    output_paths_3, id_matrix.shape[0], 3)
print(gen_probs_3.shape)
np.save(GEN_MATRIX_PATH_TEMPLATE.format(3), gen_probs_3)

number of output paths of length 3: 1203328
(18810, 18810)
